# 数据集创建&处理

In [1]:
# from modelarts.session import Session
# session = Session()
# session.obs.download_file(src_obs_file="obs://llddy/LUKE_mindspore/data/dev_data.npy", dst_local_dir="./dev_data.npy")

In [2]:
from dataset.build_dataset import build_dataset
import mindspore.dataset as ds
import os
import numpy as np
from mindspore.mindrecord import FileWriter
import json

[WARNING] ME(27598:139714374551360,MainProcess):2021-10-10-21:49:09.137.379 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install
[WARNING] ME(27598:139714374551360,MainProcess):2021-10-10-21:49:09.145.162 [mindspore/run_check/_check_version.py:181] Cuda ['10.1', '11.1'] version(need by mindspore-gpu) is not found, please confirm that the path of cuda is set to the env LD_LIBRARY_PATH, please refer to the installation guidelines: https://www.mindspore.cn/install


# 创建mindrecord

In [3]:
import numpy as np
FEATURES_FILE = "./data/dev_data1.npy"
features = np.load(FEATURES_FILE)
# print(features)

In [4]:
features

array(['{"unique_id": 1000000000, "example_index": 0, "doc_span_index": 0, "tokens": ["<s>", "\\u0120Which", "\\u0120NFL", "\\u0120team", "\\u0120represented", "\\u0120the", "\\u0120AFC", "\\u0120at", "\\u0120Super", "\\u0120Bowl", "\\u012050", "?", "</s>", "</s>", "\\u0120Super", "\\u0120Bowl", "\\u012050", "\\u0120was", "\\u0120an", "\\u0120American", "\\u0120football", "\\u0120game", "\\u0120to", "\\u0120determine", "\\u0120the", "\\u0120champion", "\\u0120of", "\\u0120the", "\\u0120National", "\\u0120Football", "\\u0120League", "\\u0120(", "NFL", ")", "\\u0120for", "\\u0120the", "\\u01202015", "\\u0120season", ".", "\\u0120The", "\\u0120American", "\\u0120Football", "\\u0120Conference", "\\u0120(", "A", "FC", ")", "\\u0120champion", "\\u0120Denver", "\\u0120Broncos", "\\u0120defeated", "\\u0120the", "\\u0120National", "\\u0120Football", "\\u0120Conference", "\\u0120(", "N", "FC", ")", "\\u0120champion", "\\u0120Carolina", "\\u0120Panthers", "\\u012024", "\\u00e2\\u0122\\u0135", "10

In [5]:
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

In [6]:
len(list_dict[0]['word_ids'])

170

In [7]:
FEATURES_FILE = "./data/dev_data.npy"
features = np.load(FEATURES_FILE)

list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad1 =  lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad1(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 256)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    "unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [-1]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    #"start_positions": {"type": "int32", "shape": [-1]},
    #"end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        "unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        #"start_positions": np.array(item["start_positions"], dtype=np.int32),
        #"end_positions": np.array(item["end_positions"], dtype=np.int32),
    }

    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

MSRStatus.SUCCESS

## 简单测试
- dev_data1.npy包含两个样本

In [8]:
FEATURES_FILE = "./data/dev_data1.npy"
features = np.load(FEATURES_FILE)





SQUAD_MINDRECORD_FILE = "./data/dev_features1.mindrecord"
list_dict = []
for item in features:
    dict_temp = json.loads(item)
    list_dict.append(dict_temp)

pad = lambda a,i : a[0:i] if len(a) > i else a + [0] * (i-len(a))
pad1 =  lambda a,i : a[0:i] if len(a) > i else a + [1] * (i-len(a))
pad_entity = lambda a,i : a[0:i] if len(a) > i else np.append(a,[-1] * (i-len(a)))

for slist in list_dict:
    slist["entity_attention_mask"] = pad(slist["entity_attention_mask"], 24)
    slist["entity_ids"] = pad(slist["entity_ids"], 24)
    slist["entity_segment_ids"] = pad(slist["entity_segment_ids"], 24)
    
    slist["word_ids"] = pad1(slist["word_ids"], 256)
    slist["word_segment_ids"] = pad(slist["word_segment_ids"], 256)
    slist["word_attention_mask"] = pad(slist["word_attention_mask"], 256)
    slist["entity_position_ids"] = np.array(slist["entity_position_ids"]).flatten()
    slist["entity_position_ids"] = pad_entity(slist["entity_position_ids"], 256)


if os.path.exists(SQUAD_MINDRECORD_FILE):
    os.remove(SQUAD_MINDRECORD_FILE)
    os.remove(SQUAD_MINDRECORD_FILE + ".db")

writer = FileWriter(file_name=SQUAD_MINDRECORD_FILE, shard_num=1)

data_schema = {
    "unique_id": {"type": "int32", "shape": [-1]},
    "word_ids": {"type": "int32", "shape": [-1]},
    "word_segment_ids": {"type": "int32", "shape": [-1]},
    "word_attention_mask": {"type": "int32", "shape": [-1]},
    "entity_ids": {"type": "int32", "shape": [-1]},
    "entity_position_ids": {"type": "int32", "shape": [-1]},
    "entity_segment_ids": {"type": "int32", "shape": [-1]},
    "entity_attention_mask": {"type": "int32", "shape": [-1]},
    #"start_positions": {"type": "int32", "shape": [-1]},
    #"end_positions": {"type": "int32", "shape": [-1]}
}
writer.add_schema(data_schema, "it is a preprocessed squad dataset")

data = []
i = 0
for item in list_dict:
    i += 1
    sample = {
        "unique_id": np.array(item["unique_id"], dtype=np.int32),
        "word_ids": np.array(item["word_ids"], dtype=np.int32),
        "word_segment_ids": np.array(item["word_segment_ids"], dtype=np.int32),
        "word_attention_mask": np.array(item["word_attention_mask"], dtype=np.int32),
        "entity_ids": np.array(item["entity_ids"], dtype=np.int32),
        "entity_position_ids": np.array(item["entity_position_ids"], dtype=np.int32),
        "entity_segment_ids": np.array(item["entity_segment_ids"], dtype=np.int32),
        "entity_attention_mask": np.array(item["entity_attention_mask"], dtype=np.int32),
        #"start_positions": np.array(item["start_positions"], dtype=np.int32),
        #"end_positions": np.array(item["end_positions"], dtype=np.int32),
    }
    
    data.append(sample)
    #print(sample)
    if i % 10 == 0:
        writer.write_raw_data(data)
        data = []

if data:
    writer.write_raw_data(data)

writer.commit()

MSRStatus.SUCCESS

In [9]:
output_features = np.array(list_dict)

In [10]:
#SQUAD_MINDRECORD_FILE = "./data/dev_features.mindrecord"
data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
count = 0
for item in data_set.create_dict_iterator():
    #print(item)
    count += 1
print("Got {} samples".format(count))

Got 2 samples


In [11]:
data_set = data_set.batch(1)
data_sample = next(data_set.create_dict_iterator())
data_sample

{'entity_attention_mask': Tensor(shape=[1, 24], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0]]),
 'entity_ids': Tensor(shape=[1, 24], dtype=Int32, value=
 [[1989, 1989,   47 ...    0,    0,    0]]),
 'entity_position_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[ 8,  9, -1 ... -1, -1, -1]]),
 'entity_segment_ids': Tensor(shape=[1, 24], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0]]),
 'unique_id': Tensor(shape=[1, 1], dtype=Int32, value=
 [[1000000001]]),
 'word_attention_mask': Tensor(shape=[1, 256], dtype=Int32, value=
 [[1, 1, 1 ... 0, 0, 0]]),
 'word_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[   0, 6834, 1485 ...    1,    1,    1]]),
 'word_segment_ids': Tensor(shape=[1, 256], dtype=Int32, value=
 [[0, 0, 0 ... 0, 0, 0]])}

In [12]:
data_sample['entity_attention_mask']

Tensor(shape=[1, 24], dtype=Int32, value=
[[1, 1, 1 ... 0, 0, 0]])

# Model

In [13]:
from readingcomprehension.models.luke import LukeForReadingComprehension, LukeEntityAwareAttentionModel
import mindspore.common.dtype as mstype
from model.bert_model import BertConfig
from mindspore import context
from model.luke import LukeModel, EntityAwareEncoder
from mindspore import Tensor, context
from mindspore import dtype as mstype
import mindspore.ops as ops
import mindspore.nn as nn
from model.bert_model import BertOutput
from mindspore.common.initializer import TruncatedNormal
from mindspore.ops import composite as C
import mindspore
from mindspore.ops import operations as P
from mindspore.train.model import Model
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

## 简单测试

In [14]:
# config = BertConfig()
# Luke_model = LukeForReadingComprehension(config)
# model = Model(Luke_model)

In [15]:
#logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)

In [16]:
#logits

# do_eval

In [17]:
!pip install tqdm
from tqdm import tqdm

In [18]:
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.model import Model
import collections
def do_eval(dataset=None, load_checkpoint_path="", eval_batch_size=1):
    config = BertConfig()
    Luke_model = LukeForReadingComprehension(config)
    Luke_model.set_train(False)
    model = Model(Luke_model)
    param_dict = load_checkpoint(load_checkpoint_path)
    load_param_into_net(Luke_model, param_dict)
    output = []
    #model = Model(Luke_model)
    
    RawResult = collections.namedtuple("RawResult", ["unique_id", "start_logits", "end_logits"])
    columns_list = ["unique_id", "word_ids", "word_segment_ids", "word_attention_mask", "entity_ids", "entity_position_ids", "entity_segment_ids", "entity_attention_mask"]
    data_set = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
    data_set = data_set.batch(eval_batch_size,drop_remainder=True)
    for data in tqdm(data_set.create_dict_iterator(num_epochs=1)):
        input_data = []
        for i in columns_list:
            input_data.append(data[i])

        unique_id, word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask = input_data
        #print(unique_id)
        logits = model.predict(word_ids,word_segment_ids,word_attention_mask,entity_ids,entity_position_ids,entity_segment_ids,entity_attention_mask)
        ids = unique_id.asnumpy()
        start = logits[0].asnumpy()
        end = logits[1].asnumpy()

        for i in range(eval_batch_size):
            unique_id = int(ids[i])
            start_logits = [float(x) for x in start[i].flat]
            end_logits = [float(x) for x in end[i].flat]
            output.append(RawResult(
                unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits))
        #print(word_ids.shape)
    return output

In [19]:
dev_dataset = ds.MindDataset(dataset_file=SQUAD_MINDRECORD_FILE)
outputs = do_eval(dev_dataset, load_checkpoint_path = "./luke-large-qa.ckpt", eval_batch_size = 1)

2it [00:16,  8.47s/it]

entity_embeddings:
Tensor(shape=[1, 24, 1024], dtype=Float32, value=
[[[-1.89387724e-01 -2.01878011e-01 -8.19541886e-03 ... -1.33859947e-01  1.38847962e-01  5.43414615e-02]
  [-1.89387724e-01 -2.01878011e-01 -8.19541886e-03 ... -1.33859947e-01  1.38847962e-01  5.43414615e-02]
  [-1.17574617e-01 -1.46372229e-01 -3.23320478e-01 ...  3.13884318e-02 -1.67741656e-01  2.38857895e-01]
  ...
  [ 8.52541625e-02  9.31130946e-02  1.75407112e-01 ... -1.30646676e-01  4.19128835e-02 -4.37492132e-03]
  [ 8.52541625e-02  9.31130946e-02  1.75407112e-01 ... -1.30646676e-01  4.19128835e-02 -4.37492132e-03]
  [ 8.52541625e-02  9.31130946e-02  1.75407112e-01 ... -1.30646676e-01  4.19128835e-02 -4.37492132e-03]]])
position_embeddings:
Tensor(shape=[1, 1024], dtype=Float32, value=
[[ 7.39479065e-03 -1.65031441e-02 -2.86699291e-02 ... -2.11757664e-02  9.61360894e-03  1.21736526e-02]])
token_type_embeddings:
Tensor(shape=[1, 24, 1024], dtype=Float32, value=
[[[ 2.01263428e-02  6.40258789e-02  4.71191406e-02 ..

In [20]:
import numpy as np
np.save('dev_outputs.npy',outputs)

# 验证

In [21]:
from model.eval_squad import read_squad_examples, write_predictions, SQuad_postprocess

In [22]:
output = np.load("./dev_outputs.npy", allow_pickle=True)
eval_examples = read_squad_examples("./dataset/dev-v1.1.json", False)
all_predictions = write_predictions(eval_examples, output_features, output, 20, 30, True)
SQuad_postprocess("./dataset/dev-v1.1.json", all_predictions, output_metrics="output.json")

f1: 0.006307158625039419
{"exact_match": 0.0, "f1": 0.006307158625039419}


In [23]:
outputs

[RawResult(unique_id=1000000001, start_logits=[-4.276713848114014, -8.810235977172852, -8.599676132202148, -9.223840713500977, -9.207724571228027, -8.721962928771973, -8.454793930053711, -9.007298469543457, -8.599067687988281, -8.913847923278809, -8.99968147277832, -8.918403625488281, -7.708374977111816, -8.416435241699219, -3.412214756011963, -5.751665115356445, -4.346570014953613, -7.040140151977539, -6.737763404846191, -6.270437717437744, -6.57470703125, -6.900035858154297, -7.225689888000488, -7.119087219238281, -7.0028510093688965, -6.307922840118408, -8.159683227539062, -6.662516117095947, -4.997922420501709, -6.7552289962768555, -6.930174350738525, -6.223684310913086, -5.52601432800293, -7.7745490074157715, -7.222416877746582, -6.49743127822876, -3.812750816345215, -6.574032306671143, -6.3123393058776855, -2.614356517791748, -3.3495774269104004, -6.28718900680542, -5.966801643371582, -5.431337356567383, -3.9703469276428223, -5.753850936889648, -6.797931671142578, -4.460217475891

In [24]:
outputs[0][0]

1000000001

In [25]:
outputs[0][1]

[-4.276713848114014,
 -8.810235977172852,
 -8.599676132202148,
 -9.223840713500977,
 -9.207724571228027,
 -8.721962928771973,
 -8.454793930053711,
 -9.007298469543457,
 -8.599067687988281,
 -8.913847923278809,
 -8.99968147277832,
 -8.918403625488281,
 -7.708374977111816,
 -8.416435241699219,
 -3.412214756011963,
 -5.751665115356445,
 -4.346570014953613,
 -7.040140151977539,
 -6.737763404846191,
 -6.270437717437744,
 -6.57470703125,
 -6.900035858154297,
 -7.225689888000488,
 -7.119087219238281,
 -7.0028510093688965,
 -6.307922840118408,
 -8.159683227539062,
 -6.662516117095947,
 -4.997922420501709,
 -6.7552289962768555,
 -6.930174350738525,
 -6.223684310913086,
 -5.52601432800293,
 -7.7745490074157715,
 -7.222416877746582,
 -6.49743127822876,
 -3.812750816345215,
 -6.574032306671143,
 -6.3123393058776855,
 -2.614356517791748,
 -3.3495774269104004,
 -6.28718900680542,
 -5.966801643371582,
 -5.431337356567383,
 -3.9703469276428223,
 -5.753850936889648,
 -6.797931671142578,
 -4.46021747589

In [26]:
outputs[0][2]

[-3.0774283409118652,
 -9.017148971557617,
 -8.657176971435547,
 -8.32359790802002,
 -8.842337608337402,
 -8.936736106872559,
 -7.790960311889648,
 -9.128320693969727,
 -9.261792182922363,
 -8.781986236572266,
 -8.156317710876465,
 -8.310914993286133,
 -8.049850463867188,
 -6.674772262573242,
 -6.908083915710449,
 -5.558197975158691,
 -4.150749206542969,
 -7.362767219543457,
 -8.157870292663574,
 -8.008962631225586,
 -7.653583526611328,
 -7.375840187072754,
 -8.504209518432617,
 -7.402853012084961,
 -8.318631172180176,
 -6.086589813232422,
 -7.869178771972656,
 -7.814949989318848,
 -7.082176208496094,
 -6.392317771911621,
 -5.168144226074219,
 -7.833662033081055,
 -5.819828033447266,
 -5.96082878112793,
 -7.870170593261719,
 -7.47831916809082,
 -4.966657638549805,
 -4.9297943115234375,
 -4.528908729553223,
 -6.072333335876465,
 -6.170160293579102,
 -6.798480987548828,
 -4.6219635009765625,
 -6.885693550109863,
 -6.768338203430176,
 -4.504144668579102,
 -4.558745384216309,
 -4.907924652

In [27]:
def evaluate(args, outputs, prefix=""):
    dataloader, examples, features, processor = load_examples(args, evaluate=True)
    all_results = []
    for batch in tqdm(dataloader, desc="eval"):
        model.eval()
        inputs = {k: v.to(args.device) for k, v in batch.items() if k != "example_indices"}
        with torch.no_grad():
            outputs = model(**inputs)

        for i, example_index in enumerate(batch["example_indices"]):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
            start_logits, end_logits = [o[i].detach().cpu().tolist() for o in outputs]
            all_results.append(Result(unique_id, start_logits, end_logits))

    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))
    if args.with_negative:
        output_null_log_odds_file = os.path.join(args.output_dir, "null_odds_{}.json".format(prefix))
    else:
        output_null_log_odds_file = None

    do_lower_case = False
    if isinstance(args.tokenizer, BertTokenizer):
        do_lower_case = args.tokenizer.basic_tokenizer.do_lower_case

    write_predictions(
        examples,
        features,
        all_results,
        args.n_best_size,
        args.max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,
        args.with_negative,
        args.null_score_diff_threshold,
        args.tokenizer,
    )

    return evaluate_on_squad(
        SQUAD_EVAL_OPTS(
            os.path.join(args.data_dir, processor.dev_file),
            pred_file=output_prediction_file,
            na_prob_file=output_null_log_odds_file,
        )
    )

In [28]:
output = np.load("./ori_dev_outputs.npy", allow_pickle=True)


In [29]:
output[1][0]

1000000001

In [30]:
output[1][1]

[-4.4323201179504395,
 -8.964935302734375,
 -8.820539474487305,
 -9.329163551330566,
 -9.370067596435547,
 -8.981746673583984,
 -8.642701148986816,
 -9.305999755859375,
 -9.00568962097168,
 -9.289281845092773,
 -9.311040878295898,
 -9.182849884033203,
 -7.984556198120117,
 -8.673095703125,
 -3.7839879989624023,
 -6.128828048706055,
 -4.672014236450195,
 -7.2613019943237305,
 -6.976904392242432,
 -6.527482509613037,
 -6.910027503967285,
 -7.155287742614746,
 -7.480282783508301,
 -7.316462516784668,
 -7.210264682769775,
 -6.343495845794678,
 -8.241094589233398,
 -6.8770751953125,
 -5.413146018981934,
 -7.053086757659912,
 -7.036792755126953,
 -6.498143196105957,
 -5.4377946853637695,
 -7.857357025146484,
 -7.398036479949951,
 -6.669647216796875,
 -3.8741912841796875,
 -6.709987163543701,
 -6.445225238800049,
 -2.77791690826416,
 -3.266592502593994,
 -6.7049150466918945,
 -6.083833694458008,
 -5.840725421905518,
 -4.378942012786865,
 -5.735772609710693,
 -7.121236801147461,
 -4.5243892669

In [31]:
output[1][2]

[-3.145303726196289,
 -8.812018394470215,
 -8.500740051269531,
 -8.169083595275879,
 -8.6859712600708,
 -8.798686027526855,
 -7.76164436340332,
 -8.993438720703125,
 -9.214651107788086,
 -8.738655090332031,
 -8.110384941101074,
 -8.12219524383545,
 -7.969758033752441,
 -6.730898857116699,
 -7.039148330688477,
 -5.615654945373535,
 -4.021191596984863,
 -7.314604759216309,
 -7.912966728210449,
 -7.804542541503906,
 -7.6259002685546875,
 -7.286072731018066,
 -8.333789825439453,
 -7.257427215576172,
 -8.123679161071777,
 -5.896554946899414,
 -7.6737165451049805,
 -7.49860954284668,
 -6.9569244384765625,
 -6.364849090576172,
 -5.064499855041504,
 -7.6801605224609375,
 -5.372745513916016,
 -5.75118350982666,
 -7.677943229675293,
 -7.222896575927734,
 -4.723443984985352,
 -4.889883995056152,
 -4.366900444030762,
 -5.9734649658203125,
 -5.968400955200195,
 -6.608968734741211,
 -4.423928260803223,
 -6.72910213470459,
 -6.716494560241699,
 -4.3286590576171875,
 -4.483022689819336,
 -4.7270584106